In [3]:
import sys

sys.path.insert(0, '../')
from nnsight import LanguageModel
import pandas as pd
import torch as t
import torch.nn as nn
from attribution import patching_effect
from dictionary_learning import AutoEncoder, ActivationBuffer
from dictionary_learning.interp import examine_dimension
from dictionary_learning.utils import zst_to_generator

In [18]:
device = 'cuda:0'

model = LanguageModel('EleutherAI/pythia-70m-deduped', device_map=device)

SEED = 42

In [19]:
def data_ambiguous(split='train', seed=SEED):
    t.manual_seed(seed)
    data = pd.read_csv(f'data/{split}_data.csv')
    labels = t.randint(0, 2, (len(data),)).to(device)
    inputs = [
        row['singular'].lower() if label == 0 else row['plural'].upper() for (_, row), label in zip(data.iterrows(), labels)
    ]
    return inputs, labels

def data_unambiguous(split='train', seed=SEED):
    t.manual_seed(seed)
    data = pd.read_csv(f'data/{split}_data.csv')
    feat1_labels = t.randint(0, 2, (len(data),)).to(device)
    feat2_labels = t.randint(0, 2, (len(data),)).to(device)
    inputs = []
    for (_, row), label1, label2 in zip(data.iterrows(), feat1_labels, feat2_labels):
        if label1 == 0 and label2 == 0:
            inputs.append(row['singular'])
        elif label1 == 0 and label2 == 1:
            inputs.append(row['singular'].upper())
        elif label1 == 1 and label2 == 0:
            inputs.append(row['plural'])
        elif label1 == 1 and label2 == 1:
            inputs.append(row['plural'].upper())
    return inputs, feat1_labels, feat2_labels

def data_gen(ambiguous=True, split='train', seed=SEED):
    if ambiguous:
        return data_ambiguous(split=split, seed=seed)
    else:
        return data_unambiguous(split=split, seed=seed)

In [20]:
def run_with_ablations(
        model,
        inputs,
        submodules,
        dictionaries,
        to_ablate,
        out_fn,
        inference=True,
):
    with model.invoke(inputs, fwd_args={'inference': inference}):
        for submodule, dictionary in zip(submodules, dictionaries):
            x = submodule.output
            is_resid = (type(x.shape) == tuple)
            if is_resid:
                x = x[0]
            x_hat = dictionary(x)
            residual = x - x_hat

            f = dictionary.encode(x)
            ablation_idxs = t.Tensor(to_ablate[submodule]).long()
            f[:, :, ablation_idxs] = 0.
            x_hat = dictionary.decode(f)
            if is_resid:
                submodule.output[0][:] = x_hat + residual
            else:
                submodule.output = x_hat + residual
            
        out = out_fn(model).save()
    return out.value

In [21]:
class Probe(nn.Module):
    def __init__(self, activation_dim):
        super().__init__()
        self.net = nn.Linear(activation_dim, 1, bias=True)

    def forward(self, x):
        logits = self.net(x).squeeze(-1)
        return logits.sigmoid()

In [22]:
lr = 1e-2
epochs = 20
layer = 2

In [23]:
inputs, feat1_labels, feat2_labels = data_gen(ambiguous=False, split='train')

with model.invoke(inputs):
    acts = model.gpt_neox.layers[layer].output[0][:,-1,:].save()
acts = acts.value.clone()

probe1 = Probe(acts.shape[-1]).to(device)
probe2 = Probe(acts.shape[-1]).to(device)
opt1 = t.optim.AdamW(probe1.parameters(), lr=lr)
opt2 = t.optim.AdamW(probe2.parameters(), lr=lr)

for _ in range(epochs):
    opt1.zero_grad(), opt2.zero_grad()
    logits1 = probe1(acts)
    logits2 = probe2(acts)
    loss1 = nn.BCELoss()(logits1, feat1_labels.float())
    loss2 = nn.BCELoss()(logits2, feat2_labels.float())
    loss1.backward(), loss2.backward()
    opt1.step(), opt2.step()

inputs, feat1_labels, feat2_labels = data_gen(ambiguous=False, split='test')

with model.invoke(inputs):
    acts = model.gpt_neox.layers[layer].output[0][:,-1,:].save()
acts = acts.value.clone()

probs1 = probe1(acts)
probs2 = probe2(acts)
preds1, preds2 = probs1.round(), probs2.round()
acc1 = (preds1 == feat1_labels).float().mean().item()
acc2 = (preds2 == feat2_labels).float().mean().item()

print(f'Control probe 1 accuracy: {acc1}')
print(f'Control probe 2 accuracy: {acc2}')


You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Control probe 1 accuracy: 0.9635036587715149
Control probe 2 accuracy: 0.9781022071838379


In [24]:
t.manual_seed(SEED)
probe = Probe(512).to(device)
optimizer = t.optim.AdamW(probe.parameters(), lr=lr)

# train probe on ambiguous data
inputs, labels = data_gen(ambiguous=True, split='train')
with model.invoke(inputs):
    acts = model.gpt_neox.layers[layer].output[0][:,-1,:].save()
acts = acts.value.clone()

for _ in range(epochs):
    optimizer.zero_grad()
    probs = probe(acts)
    loss = nn.BCELoss()(probs, labels.float())
    loss.backward()
    optimizer.step()

# get accuracy on ambiguous test set
inputs, labels = data_gen(ambiguous=True, split='test')
with model.invoke(inputs):
    acts = model.gpt_neox.layers[layer].output[0][:,-1,:].save()
acts = acts.value.clone()
preds = probe(acts).round()
acc = (preds == labels).float().mean().item()
print(f'Accuracy on ambiguous data: {acc}')

# get accuracy on unambiguous test set
inputs, feat1_labels, feat2_labels = data_gen(ambiguous=False, split='test')
with model.invoke(inputs):
    acts = model.gpt_neox.layers[layer].output[0][:,-1,:].save()
acts = acts.value.clone()
preds = probe(acts).round()
acc = (preds == feat1_labels).float().mean().item()
print(f'feat1 accuracy: {acc}')
preds = probe(acts).round()
acc = (preds == feat2_labels).float().mean().item()
print(f'feat2 accuracy: {acc}')

Accuracy on ambiguous data: 1.0
feat1 accuracy: 0.5839415788650513
feat2 accuracy: 0.9635036587715149


In [27]:
submodules = [
    model.gpt_neox.layers[i] for i in range(layer + 1)
]
dictionaries = []
for i in range(layer + 1):
    ae = AutoEncoder(512, 64 * 512).to(device)
    ae.load_state_dict(t.load(f'/share/projects/dictionary_circuits/autoencoders/pythia-70m-deduped/resid_out_layer{i}/0_32768/ae.pt'))
    dictionaries.append(ae)

def metric_fn(model):
    return probe(model.gpt_neox.layers[layer].output[0][:,-1,:])

inputs, labels = data_gen(ambiguous=True, split='train')

neg_inputs, pos_inputs = [], []
for x, label in zip(inputs, labels):
    if label == 0:
        neg_inputs.append(x)
    else:
        pos_inputs.append(x)


neg_effects, _ = patching_effect(
    neg_inputs,
    None,
    model,
    submodules,
    dictionaries,
    metric_fn,
    method='ig'
)

neg_effects = {k : v.mean(dim=0).mean(dim=0) for k, v in neg_effects.items()}
for i, submodule in enumerate(submodules):
    print(f"Layer {i}:")
    effect = neg_effects[submodule]
    for feature_idx in t.nonzero(effect):
        value = effect[tuple(feature_idx)]
        if value > 0.001:
            print(f"    Multindex: {tuple(feature_idx.tolist())}, Value: {value}")

pos_effects, _ = patching_effect(
    pos_inputs,
    None,
    model,
    submodules,
    dictionaries,
    metric_fn,
    method='ig'
)

pos_effects = {k : v.mean(dim=0).mean(dim=0) for k, v in pos_effects.items()}

for i, submodule in enumerate(submodules):
    print(f"Layer {i}:")
    effect = pos_effects[submodule]
    for feature_idx in t.nonzero(effect):
        value = effect[tuple(feature_idx)]
        if -value > 0.001:
            print(f"    Multindex: {tuple(feature_idx.tolist())}, Value: {value}")

# total_effects = {
#     k : -pos_effects[k] + neg_effects[k] for k in pos_effects.keys()
# }
# for i, submodule in enumerate(submodules):
#     print(f"Layer {i}:")
#     effect = total_effects[submodule]
#     for feature_idx in t.nonzero(effect):
#         value = effect[tuple(feature_idx)]
#         if value > 0.001:
#             print(f"    Multindex: {tuple(feature_idx.tolist())}, Value: {value}")

Layer 0:
    Multindex: (23,), Value: 0.001021671574562788
    Multindex: (670,), Value: 0.0018523469334468246
    Multindex: (4324,), Value: 0.009785940870642662
    Multindex: (7466,), Value: 0.0032092721667140722
    Multindex: (15637,), Value: 0.0031055279541760683
    Multindex: (18773,), Value: 0.0024783434346318245
    Multindex: (20334,), Value: 0.0026556900702416897
    Multindex: (22464,), Value: 0.0010544315446168184
Layer 1:
    Multindex: (7633,), Value: 0.0019621977116912603
    Multindex: (12833,), Value: 0.007209473289549351
    Multindex: (15714,), Value: 0.001227965927682817
    Multindex: (19812,), Value: 0.00587422912940383
Layer 2:
    Multindex: (16421,), Value: 0.0016992193413898349
    Multindex: (17187,), Value: 0.0012213450390845537
Layer 0:
    Multindex: (5493,), Value: -0.001290997490286827
    Multindex: (5650,), Value: -0.002565438859164715
    Multindex: (10316,), Value: -0.0012732871109619737
    Multindex: (13114,), Value: -0.0017375940224155784
    Mu

In [319]:
component_idx = 1
feat_idx = 19812

submodule = submodules[component_idx]
dictionary = dictionaries[component_idx]

# interpret some features
data = zst_to_generator('/share/data/datasets/pile/the-eye.eu/public/AI/pile/train/00.jsonl.zst')
buffer = ActivationBuffer(
    data,
    model,
    submodule,
    out_feats=512,
    in_batch_size=128,
    n_ctxs=512,
    is_hf=False
)

out = examine_dimension(
    model,
    submodule,
    buffer,
    dictionary,
    dim_idx=feat_idx,
)
print(out['top_tokens'])
out['top_contexts']

[('def', 8.276412010192871), ('ive', 8.197701454162598), ('ised', 7.696048736572266), (' Alternative', 6.6908488273620605), ('<|endoftext|>', 6.6041448035340675), ('--', 6.535338401794434), ('abc', 5.8198747634887695), ('recurs', 5.672883987426758), ('PCI', 4.438255310058594), ('Standard', 2.646510124206543)]


In [28]:
input = "YARNS"

to_ablate = {
    submodules[0] : [
        5493,
        5650,
        13114,
        # 22182, # feature for tokens containing 'S'
        25864,
        # 26261, # another 'S' feature
        # 26532, # feature for things starting with 'S'
    ],
    submodules[1] : [
        2779,
        # 4900, # plural words that start with a capital letter
        5469,
        5509,
        7368,
        # 11305, # sometimes fires on other stuff
        # 14465, # all caps bigrams ending in 'S'
        18471,
        # 25963, # for the 'S' token
        26424,
        32267,  
    ],
    submodules[2] : [
        # 9068, # for the 'S' token
        22968,
        27888,
    ]
}


with model.invoke(input):
    acts = model.gpt_neox.layers[layer].output[0][:,-1,:].save()
acts = acts.value.clone()
pred_before = probe(acts).item()

with model.invoke(input):
    for submodule, dictionary in zip(submodules, dictionaries):
        x = submodule.output
        is_resid = type(x.shape) == tuple
        if is_resid:
            x = x[0]
        x_hat = dictionary(x)
        residual = x - x_hat
        
        f = dictionary.encode(x)
        ablation_idxs = to_ablate[submodule]
        for idx in ablation_idxs:
            f[..., idx] = 0
        x_hat = dictionary.decode(f)
        if is_resid:
            submodule.output[0][:] = x_hat + residual
        else:
            submodule.output = x_hat + residual
    acts = model.gpt_neox.layers[layer].output[0][:,-1,:].save()
acts = acts.value.clone()
pred_after = probe(acts).item()

print(f'before: {pred_before}, after: {pred_after}')

before: 0.7661039233207703, after: 0.38638415932655334


In [29]:
def get_acts(model):
    return model.gpt_neox.layers[layer].output[0][:,-1,:]

# get accuracy on ambiguous test set
inputs, labels = data_gen(ambiguous=True, split='test')
acts = run_with_ablations(
    model,
    inputs,
    submodules,
    dictionaries,
    to_ablate,
    get_acts,
)
preds = probe(acts.clone()).round()
acc = (preds == labels).float().mean().item()
print(f'Accuracy on ambiguous data: {acc}')

# get accuracy on unambiguous test set
inputs, feat1_labels, feat2_labels = data_gen(ambiguous=False, split='test')
acts = run_with_ablations(
    model,
    inputs,
    submodules,
    dictionaries,
    to_ablate,
    get_acts,
)
preds = probe(acts.clone()).round()
acc = (preds == feat1_labels).float().mean().item()
print(f'feat1 accuracy: {acc}')
preds = probe(acts.clone()).round()
acc = (preds == feat2_labels).float().mean().item()
print(f'feat2 accuracy: {acc}')


Accuracy on ambiguous data: 0.9781022071838379
feat1 accuracy: 0.6569343209266663
feat2 accuracy: 0.8613138794898987


In [30]:
# train probes on unambiguous data
inputs, feat1_labels, feat2_labels = data_gen(ambiguous=False, split='train')

acts = run_with_ablations(
    model,
    inputs,
    submodules,
    dictionaries,
    to_ablate,
    get_acts,
)
acts = acts.clone()

t.manual_seed(SEED)
probe1 = Probe(acts.shape[-1]).to(device)
probe2 = Probe(acts.shape[-1]).to(device)
opt1 = t.optim.AdamW(probe1.parameters(), lr=lr)
opt2 = t.optim.AdamW(probe2.parameters(), lr=lr)

for _ in range(epochs):
    opt1.zero_grad(), opt2.zero_grad()
    logits1 = probe1(acts)
    logits2 = probe2(acts)
    loss1 = nn.BCELoss()(logits1, feat1_labels.float())
    loss2 = nn.BCELoss()(logits2, feat2_labels.float())
    loss1.backward(), loss2.backward()
    opt1.step(), opt2.step()

# get accuracy on unambiguous test set
inputs, feat1_labels, feat2_labels = data_gen(ambiguous=False, split='test')
acts = run_with_ablations(
    model,
    inputs,
    submodules,
    dictionaries,
    to_ablate,
    get_acts,
)
acts = acts.clone()

probs1 = probe1(acts)
probs2 = probe2(acts)
preds1, preds2 = probs1.round(), probs2.round()
acc1 = (preds1 == feat1_labels).float().mean().item()
acc2 = (preds2 == feat2_labels).float().mean().item()

print(f'feat1 accuracy: {acc1}')
print(f'feat2 accuracy: {acc2}')


feat1 accuracy: 0.956204354763031
feat2 accuracy: 0.970802903175354


In [31]:
# retrain probe on ablated model
t.manual_seed(SEED)
probe_new = Probe(512).to(device)
optimizer = t.optim.AdamW(probe_new.parameters(), lr=lr)

# train probe on ambiguous data
inputs, labels = data_gen(ambiguous=True, split='train')
acts = run_with_ablations(
    model,
    inputs,
    submodules,
    dictionaries,
    to_ablate,
    get_acts,
)
acts = acts.clone()

for _ in range(epochs):
    optimizer.zero_grad()
    probs = probe_new(acts)
    loss = nn.BCELoss()(probs, labels.float())
    loss.backward()
    optimizer.step()

# get accuracy on ambiguous test set
inputs, labels = data_gen(ambiguous=True, split='test')
acts = run_with_ablations(
    model,
    inputs,
    submodules,
    dictionaries,
    to_ablate,
    get_acts,
)
acts = acts.clone()
preds = probe_new(acts).round()
acc = (preds == labels).float().mean().item()
print(f'Accuracy on ambiguous data: {acc}')

# get accuracy on unambiguous test set
inputs, feat1_labels, feat2_labels = data_gen(ambiguous=False, split='test')
acts = run_with_ablations(
    model,
    inputs,
    submodules,
    dictionaries,
    to_ablate,
    get_acts,
)
acts = acts.clone()
preds = probe_new(acts).round()
acc = (preds == feat1_labels).float().mean().item()
print(f'feat1 accuracy: {acc}')
preds = probe_new(acts).round()
acc = (preds == feat2_labels).float().mean().item()
print(f'feat2 accuracy: {acc}')

Accuracy on ambiguous data: 0.985401451587677
feat1 accuracy: 0.6788321137428284
feat2 accuracy: 0.8540145754814148


In [32]:
# classify things based on the difference between the probe and probe_new predictions
inputs, feat1_labels, feat2_labels = data_gen(ambiguous=False, split='test')

with model.invoke(inputs):
    acts = model.gpt_neox.layers[layer].output[0][:,-1,:].save()
acts = acts.value.clone()

acts_new = run_with_ablations(
    model,
    inputs,
    submodules,
    dictionaries,
    to_ablate,
    get_acts,
)
acts_new = acts_new.clone()

probs1 = probe(acts)
probs2 = probe_new(acts_new)
diffs = probs2 - probs1
preds = t.where(
    diffs.abs() > .05,
    (diffs > 0).float(),
    probs2.round()
)

acc = (preds == feat1_labels).float().mean().item()
print(f'feat1 accuracy: {acc}')
acc = (preds == feat2_labels).float().mean().item()
print(f'feat2 accuracy: {acc}')

feat1 accuracy: 0.7591241002082825
feat2 accuracy: 0.5985401272773743


In [372]:
for x, prob1, prob2, pred in zip(inputs, probs1, probs2, preds):
    print(x, pred.item(), prob1.item(), prob2.item())

beef 0.0 0.04246163368225098 0.0424615778028965
tomatoes 0.0 0.01407528854906559 0.012785992585122585
headphone 0.0 0.0004125593404751271 0.00041255910764448345
ELECTRON 0.0 0.9458839893341064 0.4734917879104614
seatbelt 0.0 0.002296003745868802 0.002296007238328457
GOBLINS 0.0 0.9927135109901428 0.919037938117981
TRAVELER 0.0 0.9688680768013 0.4352923631668091
DANCE 0.0 0.35750260949134827 0.12069746106863022
parrot 0.0 0.0020670697558671236 0.002028270158916712
minerals 0.0 0.06108663231134415 0.06108665466308594
refugee 0.0 0.09666279703378677 0.09666275233030319
cabinet 0.0 0.015743708238005638 0.014384628273546696
RUG 0.0 0.6748154759407043 0.4169016480445862
doctor 0.0 0.00024147509247995913 0.00024147509247995913
cookies 0.0 0.037880077958106995 0.03496574983000755
bicycle 0.0 0.019201841205358505 0.018249278888106346
toothbrushes 0.0 0.21269121766090393 0.2057013362646103
zoos 0.0 0.05740008503198624 0.056083958595991135
ROADS 0.0 0.539910614490509 0.3340230882167816
butter 0.0

In [340]:
probs2 - probs1

tensor([ 3.6879e-02,  2.0250e-02, -1.3686e-04,  2.3178e-01,  1.6529e-03,
         6.6891e-02,  2.7669e-01,  1.0507e-01, -2.0387e-04,  9.0288e-02,
         1.3871e-01,  2.7971e-02,  2.8520e-01, -1.7768e-04,  2.1047e-01,
         5.7020e-02,  4.4652e-01, -2.1454e-02,  5.7701e-01,  2.8447e-02,
         8.5847e-02,  2.5172e-05,  1.0240e-02,  3.1668e-01,  1.2039e-01,
         3.1745e-01,  1.5877e-01,  2.4371e-03, -5.2675e-04,  1.5538e-01,
         1.2155e-01, -9.7485e-05,  2.5782e-02,  5.9511e-02,  7.9205e-02,
         2.8743e-02,  9.1319e-02,  5.9899e-02,  4.1422e-01,  2.7963e-01,
         3.3364e-02,  7.9152e-02,  7.3441e-03,  4.3555e-01, -3.0199e-04,
         6.6104e-03, -3.7750e-04, -4.0154e-04,  3.5560e-01,  5.5813e-02,
         2.9123e-04,  4.7253e-01,  2.7764e-01,  3.6436e-01, -3.0451e-02,
         7.5291e-02,  2.3076e-01,  1.6617e-01,  1.3763e-01,  3.0491e-01,
         3.5337e-01,  1.4857e-01, -3.3222e-04,  3.4590e-01,  2.2563e-02,
         4.7432e-01,  4.9966e-01,  3.7437e-01,  1.4

In [7]:
test_data = pd.read_json('/share/data/datasets/msgs/syntactic_category_lexical_content_the/test.jsonl', lines=True)

ling_accs, surface_accs = [], []
# get accuracy on test data
for batch_idx in range(len(test_data) // batch_size):
    inputs = test_data['sentence'][batch_idx * batch_size:(batch_idx + 1) * batch_size].tolist()
    ling_labels = test_data['linguistic_feature_label'][batch_idx * batch_size:(batch_idx + 1) * batch_size].tolist()
    surface_labels = test_data['surface_feature_label'][batch_idx * batch_size:(batch_idx + 1) * batch_size].tolist()

    with model.invoke(inputs) as invoker:
        hidden_states = model.gpt_neox.layers[-3].output[0].save()
    
    with t.no_grad():
        preds = probe(hidden_states.value)
        ling_acc = (preds.round() == t.Tensor(ling_labels).to('cuda:0')).float().mean()
        surface_acc = (preds.round() == t.Tensor(surface_labels).to('cuda:0')).float().mean()
        ling_accs.append(ling_acc.item())
        surface_accs.append(surface_acc.item())

print('ling acc:', sum(ling_accs) / len(ling_accs))
print('surface acc:', sum(surface_accs) / len(surface_accs))
    

ling acc: 0.9539930555555556
surface acc: 0.6663995726495726


In [8]:
from attribution import patching_effect
from dictionary_learning.dictionary import AutoEncoder

In [16]:
clean = "All grandsons do resemble the print and Debra is an organized child."
patch = "All grandsons do resemble a print and Debra is an banana child."

with model.invoke([clean, patch]) as invoker:
    hidden_states = model.gpt_neox.layers[-3].output[0].save()

with t.no_grad():
    preds = probe(hidden_states.value)
preds

tensor([0.4021, 0.0954], device='cuda:0')

In [17]:
def metric_fn(model):
    return probe(model.gpt_neox.layers[-3].output[0])

submodules = [
    model.gpt_neox.layers[i] for i in range(4)
] + [
    model.gpt_neox.layers[i].mlp for i in range(4)
]
dictionaries = []
for i in range(4):
    dictionary = AutoEncoder(512, 64 * 512).to(device)
    dictionary.load_state_dict(t.load(f'/share/projects/dictionary_circuits/autoencoders/pythia-70m-deduped/resid_out_layer{i}/0_32768/ae.pt'))
    dictionaries.append(dictionary)
for i in range(4):
    dictionary = AutoEncoder(512, 64 * 512).to(device)
    dictionary.load_state_dict(t.load(f'/share/projects/dictionary_circuits/autoencoders/pythia-70m-deduped/mlp_out_layer{i}/1_32768/ae.pt'))
    dictionaries.append(dictionary)

out = patching_effect(
    clean,
    patch,
    model,
    submodules,
    dictionaries,
    metric_fn,
)

In [19]:
effects, total_effect = out
print(f"Total effect: {total_effect}")
for layer, submodule in enumerate(submodules):
    print(f"Layer {layer}:")
    effect = effects[submodule]
    for feature_idx in t.nonzero(effect):
        value = effect[tuple(feature_idx)]
        if value.abs() > 0.1:
            print(f"    Multindex: {tuple(feature_idx.tolist())}, Value: {value}")

Total effect: tensor([-0.7627], device='cuda:0', grad_fn=<DivBackward0>)
Layer 0:
    Multindex: (0, 6, 23084), Value: -0.31019383668899536
    Multindex: (0, 6, 29115), Value: -0.2026602178812027
    Multindex: (0, 12, 9247), Value: 0.23665377497673035
    Multindex: (0, 12, 19133), Value: -0.13597454130649567
    Multindex: (0, 13, 1147), Value: 0.10391081869602203
    Multindex: (0, 13, 1256), Value: -0.12887312471866608
    Multindex: (0, 13, 3385), Value: -0.34761813282966614
    Multindex: (0, 13, 3613), Value: -0.11544839292764664
    Multindex: (0, 13, 5702), Value: -11.003226280212402
    Multindex: (0, 13, 5962), Value: -0.18838448822498322
    Multindex: (0, 13, 6959), Value: -0.9820340871810913
    Multindex: (0, 13, 15146), Value: -0.12887312471866608
    Multindex: (0, 13, 25951), Value: -0.24457168579101562
    Multindex: (0, 13, 26640), Value: -0.25634047389030457
    Multindex: (0, 13, 27692), Value: -0.12028089165687561
    Multindex: (0, 13, 31525), Value: -0.1246131

In [15]:
for i, tok in enumerate(invoker.input.input_ids[0]):
    print(f"{i}: {model.tokenizer.decode([tok])}")

0: All
1:  gr
2: ands
3: ons
4:  do
5:  resemble
6:  the
7:  print
8:  and
9:  De
10: bra
11:  is
12:  an
13:  organized
14:  child
15: .


In [ ]:
# to_ablate = {
#     submodules[0] : [
#         5650,
#         17126,
#         22182,
#         25864,
#         2655,
#         12249,
#         12267,
#         21248,
#         21329,
#     ],
#     submodules[1] : [
#         14465,
#         18471,
#         22990,
#         16629,
#         26134,
#         32267,
#     ],
#     submodules[2] : [
#         16421,
#         22968,
#         27888,
#         15899,
#         28262,
#         28306,
#         32164,
#     ]
# }